In [18]:
import pandas as pd
import numpy as np

# manually created files
divisions = pd.read_csv('C:/Users/JoshuaTerc/Documents/NFL/Division.csv')
teams = pd.read_csv('C:/Users/JoshuaTerc/Documents/NFL/Teams.csv')
games = pd.read_csv('C:/Users/JoshuaTerc/Documents/NFL/Games.csv')

games['Result'] = np.where(games['HomeTeamScore'] > games['AwayTeamScore'], 'W', np.where(games['HomeTeamScore'] < games['AwayTeamScore'], 'L', 'T'))
games['ScoreDiff'] = games['HomeTeamScore'] - games['AwayTeamScore']
games['WinningTeamId'] = np.where(games['HomeTeamScore'] > games['AwayTeamScore'], games['HomeTeamId'], np.where(games['AwayTeamScore'] > games['HomeTeamScore'], games['AwayTeamId'], np.NaN))
games['WinningTeam'] = np.where(games['HomeTeamScore'] > games['AwayTeamScore'], 'H', np.where(games['AwayTeamScore'] > games['HomeTeamScore'], 'A', np.where(games['HomeTeamScore'] == games['AwayTeamScore'], 'T', np.NaN)))
games['LosingTeamId'] = np.where(games['HomeTeamScore'] > games['AwayTeamScore'], games['AwayTeamId'], np.where(games['AwayTeamScore'] > games['HomeTeamScore'], games['HomeTeamId'], np.NaN))
games['TieHomeTeamId'] = np.where(games['HomeTeamScore'] == games['AwayTeamScore'], games['HomeTeamId'], np.NaN)
games['TieAwayTeamId'] = np.where(games['HomeTeamScore'] == games['AwayTeamScore'], games['AwayTeamId'], np.NaN)

teams = pd.merge(teams, games.groupby('WinningTeamId').count()['Id'].reset_index(name='Wins'), how = 'left', left_on = 'Id', right_on = 'WinningTeamId')
teams = pd.merge(teams, games.groupby('LosingTeamId').count()['Id'].reset_index(name='Losses'), how = 'left', left_on = 'Id', right_on = 'LosingTeamId')
teams = pd.merge(teams, games.groupby('TieHomeTeamId').count()['Id'].reset_index(name='HomeTies'), how = 'left', left_on = 'Id', right_on = 'TieHomeTeamId')
teams = pd.merge(teams, games.groupby('TieAwayTeamId').count()['Id'].reset_index(name='AwayTies'), how = 'left', left_on = 'Id', right_on = 'TieAwayTeamId')

teams['Wins'] = np.where(teams['Wins'].isnull(), 0, teams['Wins'])
teams['Losses'] = np.where(teams['Losses'].isnull(), 0, teams['Losses'])
teams['Ties'] = np.where(teams['HomeTies'].isnull(), 0,teams['HomeTies']) + np.where(teams['AwayTies'].isnull(), 0,teams['AwayTies'])
teams = teams.drop(teams.columns[[3,5,7,8,9,10]], axis=1)
teams['Pct'] = round(teams['Wins'] / (teams['Wins'] + teams['Losses']),3)
teams['DivisionStanding'] = teams.groupby('DivisionId')['Pct'].rank(method="min", ascending=False)

teams = pd.merge(teams, games[games['WinningTeam']=='H'].groupby('WinningTeamId').count()['Id'].reset_index(name='HomeWins'), how = 'left', left_on = 'Id', right_on = 'WinningTeamId')
teams = pd.merge(teams, games[games['WinningTeam']=='A'].groupby('LosingTeamId').count()['Id'].reset_index(name='HomeLosses'), how = 'left', left_on = 'Id', right_on = 'LosingTeamId')
teams = teams.drop(teams.columns[[8,10]], axis=1)

teams = pd.merge(teams, games[games['WinningTeam']=='A'].groupby('WinningTeamId').count()['Id'].reset_index(name='AwayWins'), how = 'left', left_on = 'Id', right_on = 'WinningTeamId')
teams = pd.merge(teams, games[games['WinningTeam']=='H'].groupby('LosingTeamId').count()['Id'].reset_index(name='AwayLosses'), how = 'left', left_on = 'Id', right_on = 'LosingTeamId')
teams = teams.drop(teams.columns[[10,12]], axis=1)

teams['HomeWins'] = np.where(teams['HomeWins'].isnull(), 0, teams['HomeWins'])
teams['HomeLosses'] = np.where(teams['HomeLosses'].isnull(), 0, teams['HomeLosses'])
teams['AwayWins'] = np.where(teams['AwayWins'].isnull(), 0, teams['AwayWins'])
teams['AwayLosses'] = np.where(teams['AwayLosses'].isnull(), 0, teams['AwayLosses'])
teams['HomePct'] = round(teams['HomeWins'] / (teams['HomeWins'] + teams['HomeLosses']),3)
teams['AwayPct'] = round(teams['AwayWins'] / (teams['AwayWins'] + teams['AwayLosses']),3)

games.head()

,Id,Week,HomeTeamId,AwayTeamId,Spread,HomeTeamScore,AwayTeamScore,Result,ScoreDiff,WinningTeamId,WinningTeam,LosingTeamId,TieHomeTeamId,TieAwayTeamId
0,1,1,30,9,-7.5,31,29,W,2,30.0,H,9.0,NaN,NaN
1,2,1,24,10,3.0,13,27,L,-14,10.0,A,24.0,NaN,NaN
2,3,1,2,26,3.0,6,32,L,-26,26.0,A,2.0,NaN,NaN
3,4,1,4,27,-6.5,16,32,L,-16,27.0,A,4.0,NaN,NaN
4,5,1,7,21,3.0,27,24,W,3,7.0,H,21.0,NaN,NaN


In [19]:
## calculating points scored

home_points_for = []
for i in range(1,33):
    df = games[(games['HomeTeamId']==i)][['HomeTeamId','HomeTeamScore']]
    home_points_for.append(df)
all_home_points_for = pd.concat(home_points_for)
home_points_for = all_home_points_for.groupby('HomeTeamId')['HomeTeamScore'].sum().reset_index(name = 'HomePointsFor')

away_points_for = []
for i in range(1,33):
    df = games[(games['AwayTeamId']==i)][['AwayTeamId','AwayTeamScore']]
    away_points_for.append(df)
all_away_points_for = pd.concat(away_points_for)
away_points_for = all_away_points_for.groupby('AwayTeamId')['AwayTeamScore'].sum().reset_index(name = 'AwayPointsFor')

away_points_against = []
for i in range(1,33):
    df = games[(games['AwayTeamId']==i)][['AwayTeamId','HomeTeamScore']]
    away_points_against.append(df)
all_away_points_against = pd.concat(away_points_against)
away_points_against = all_away_points_against.groupby('AwayTeamId')['HomeTeamScore'].sum().reset_index(name = 'AwayPointsAgainst')

home_points_against = []
for i in range(1,33):
    df = games[(games['HomeTeamId']==i)][['HomeTeamId','AwayTeamScore']]
    home_points_against.append(df)
all_home_points_against = pd.concat(home_points_against)
home_points_against = all_home_points_against.groupby('HomeTeamId')['AwayTeamScore'].sum().reset_index(name = 'HomePointsAgainst')

points_against = pd.merge(home_points_against, away_points_against, how = 'inner', left_on='HomeTeamId', right_on='AwayTeamId')
points_against = points_against.drop(points_against.columns[2], axis=1)
points_against = points_against.rename(columns={'HomeTeamId': 'TeamId'})
points_against['TotalPointsAgainst'] = points_against['HomePointsAgainst'] + points_against['AwayPointsAgainst']
points_against['PointsAgainstRank'] = points_against['TotalPointsAgainst'].rank(method='min', ascending=True)

points_for = pd.merge(home_points_for, away_points_for, how = 'inner', left_on='HomeTeamId', right_on='AwayTeamId')
points_for = points_for.drop(points_for.columns[2], axis=1)
points_for = points_for.rename(columns={'HomeTeamId': 'TeamId'})
points_for['TotalPointsFor'] = points_for['HomePointsFor'] + points_for['AwayPointsFor']
points_for['PointsForRank'] = points_for['TotalPointsFor'].rank(method='min', ascending=False)
points_for.head(10)

,TeamId,HomePointsFor,AwayPointsFor,TotalPointsFor,PointsForRank
0,1,182,267,449,7.0
1,2,158,171,329,26.0
2,3,230,157,387,17.0
3,4,237,249,486,3.0
4,5,145,159,304,29.0
5,6,153,173,326,27.0
6,7,231,210,441,9.0
7,8,170,179,349,20.0
8,9,291,239,530,1.0
9,10,187,148,335,23.0


In [20]:
teams = pd.merge(teams, points_for[['TeamId', 'TotalPointsFor', 'PointsForRank']], how = 'inner', left_on = 'Id', right_on = 'TeamId')
teams = pd.merge(teams, points_against[['TeamId', 'TotalPointsAgainst', 'PointsAgainstRank']], how = 'inner', on = 'TeamId')
teams['AvgRank'] = round((teams['PointsForRank'] + teams['PointsAgainstRank'])/2,2)
teams['PPGFor'] = round(teams['TotalPointsFor']/(teams['Wins'] + teams['Losses']),1)
teams['PPGAgainst'] = round(teams['TotalPointsAgainst']/(teams['Wins'] + teams['Losses']),1)
teams.head()

,Id,Team,DivisionId,Wins,Losses,Ties,Pct,DivisionStanding,HomeWins,HomeLosses,...,HomePct,AwayPct,TeamId,TotalPointsFor,PointsForRank,TotalPointsAgainst,PointsAgainstRank,AvgRank,PPGFor,PPGAgainst
0,1,Arizona Cardinals,8,11,6,0.0,0.647,2.0,3,5,...,0.375,0.889,1,449,7.0,366,11.0,9.0,26.4,21.5
1,2,Atlanta Falcons,7,6,11,0.0,0.353,3.0,3,5,...,0.375,0.333,2,329,26.0,443,28.0,27.0,19.4,26.1
2,3,Baltimore Ravens,2,8,9,0.0,0.471,3.0,5,4,...,0.556,0.375,3,387,17.0,392,19.0,18.0,22.8,23.1
3,4,Buffalo Bills,1,12,5,0.0,0.706,1.0,6,3,...,0.667,0.750,4,486,3.0,295,1.0,2.0,28.6,17.4
4,5,Carolina Panthers,7,5,12,0.0,0.294,4.0,2,6,...,0.250,0.333,5,304,29.0,404,22.0,25.5,17.9,23.8


In [21]:
wins = []
for i in range(1,33):
    df = games[(games['WinningTeamId']==i)][['WinningTeamId','LosingTeamId']]
    wins.append(df)
all_wins = pd.concat(wins)
all_wins = pd.merge(all_wins, teams[['Id','Pct']], left_on = 'LosingTeamId', right_on = 'Id', how = 'inner')
SOW = all_wins.groupby('WinningTeamId')['Pct'].mean().round(3).reset_index(name = 'SOW')
SOW = SOW.rename(columns={'WinningTeamId':'Id'})
teams = pd.merge(teams, SOW, how = 'left', on = 'Id')

losses = []
for i in range(1,33):
    df = games[(games['LosingTeamId']==i)][['WinningTeamId','LosingTeamId']]
    losses.append(df)
all_losses = pd.concat(losses)
all_losses = pd.merge(all_losses, teams[['Id','Pct']], left_on = 'WinningTeamId', right_on = 'Id', how = 'inner')
SOL = all_losses.groupby('LosingTeamId')['Pct'].mean().round(3).reset_index(name = 'SOL')
SOL = SOL.rename(columns={'LosingTeamId':'Id'})
teams = pd.merge(teams, SOL, how = 'left', on = 'Id')
teams.head()

,Id,Team,DivisionId,Wins,Losses,Ties,Pct,DivisionStanding,HomeWins,HomeLosses,...,TeamId,TotalPointsFor,PointsForRank,TotalPointsAgainst,PointsAgainstRank,AvgRank,PPGFor,PPGAgainst,SOW,SOL
0,1,Arizona Cardinals,8,11,6,0.0,0.647,2.0,3,5,...,1,449,7.0,366,11.0,9.0,26.4,21.5,0.497,0.463
1,2,Atlanta Falcons,7,6,11,0.0,0.353,3.0,3,5,...,2,329,26.0,443,28.0,27.0,19.4,26.1,0.345,0.545
2,3,Baltimore Ravens,2,8,9,0.0,0.471,3.0,5,4,...,3,387,17.0,392,19.0,18.0,22.8,23.1,0.461,0.605
3,4,Buffalo Bills,1,12,5,0.0,0.706,1.0,6,3,...,4,486,3.0,295,1.0,2.0,28.6,17.4,0.407,0.618
4,5,Carolina Panthers,7,5,12,0.0,0.294,4.0,2,6,...,5,304,29.0,404,22.0,25.5,17.9,23.8,0.412,0.544


In [24]:
# change for current week number
wk = 17

home_teams = pd.merge(games[games['Week']==wk][['Week','HomeTeamId','AwayTeamId','Spread']], teams[['Id','Team','PPGFor','PPGAgainst','SOW','SOL']], how = 'inner', left_on = 'HomeTeamId', right_on = 'Id')
home_teams = home_teams.drop(home_teams.columns[4], axis=1)
home_teams = home_teams.rename(columns={'PPGFor': 'PPGForH', 'PPGAgainst': 'PPGAgainstH', 'Team': 'HomeTeam', 'SOW': 'SOWH', 'SOL':'SOLH'})
games_week = pd.merge(home_teams, teams[['Id','Team','PPGFor','PPGAgainst','SOW','SOL']], how = 'inner', left_on = 'AwayTeamId', right_on = 'Id')
games_week = games_week.drop(games_week.columns[9], axis=1)
games_week = games_week.rename(columns={'PPGFor': 'PPGForA', 'PPGAgainst': 'PPGAgainstA', 'Team': 'AwayTeam', 'SOW': 'SOWA', 'SOL':'SOLA'})
games_week['ExpectedPtsH'] = round((games_week['PPGForH'] + games_week['PPGAgainstA'])/2,1)
games_week['ExpectedPtsA'] = round((games_week['PPGForA'] + games_week['PPGAgainstH'])/2,1)
games_week['ExpectedSpread'] = round(games_week['ExpectedPtsA'],0) - round(games_week['ExpectedPtsH'],0)
games_week['ExpectedWinner'] = np.where(games_week['ExpectedPtsH'] > games_week['ExpectedPtsA'], games_week['HomeTeam'], games_week['AwayTeam'])
games_week['ExpectedCover'] = np.where((games_week['Spread'] < 0) & (games_week['ExpectedSpread'] < games_week['Spread']), games_week['HomeTeam']
                                       , np.where((games_week['Spread'] < 0) & (games_week['ExpectedSpread'] >= games_week['Spread']), games_week['AwayTeam']
                                       , np.where((games_week['Spread'] > 0) & (games_week['ExpectedSpread'] <= games_week['Spread']), games_week['HomeTeam']
                                       , np.where((games_week['Spread'] > 0) & (games_week['ExpectedSpread'] > games_week['Spread']), games_week['AwayTeam']
                                       , np.where((games_week['Spread'] == 0) & (games_week['ExpectedSpread'] > 0), games_week['AwayTeam']
                                       , np.where((games_week['Spread'] == 0) & (games_week['ExpectedSpread'] < 0), games_week['HomeTeam'], np.NaN))))))

games_week = games_week[['Week', 'HomeTeamId', 'AwayTeamId', 'HomeTeam', 'PPGForH', 'PPGAgainstH', 'SOLH', 'SOLA', 'AwayTeam', 'PPGForA', 'PPGAgainstA', 'SOWH', 'SOWA', 'ExpectedPtsH', 'ExpectedPtsA', 'Spread', 'ExpectedSpread', 'ExpectedWinner', 'ExpectedCover']]
games_week

,Week,HomeTeamId,AwayTeamId,HomeTeam,PPGForH,PPGAgainstH,SOLH,SOLA,AwayTeam,PPGForA,PPGAgainstA,SOWH,SOWA,ExpectedPtsH,ExpectedPtsA,Spread,ExpectedSpread,ExpectedWinner,ExpectedCover
0,17,6,24,Chicago Bears,19.2,23.1,0.580,0.550,New York Giants,15.2,24.5,0.421,0.485,21.8,19.2,-6.5,-3.0,Chicago Bears,New York Giants
1,17,9,1,Dallas Cowboys,31.2,21.1,0.624,0.463,Arizona Cardinals,26.4,21.5,0.419,0.497,26.4,23.8,-6.5,-2.0,Dallas Cowboys,Arizona Cardinals
2,17,4,2,Buffalo Bills,28.6,17.4,0.618,0.545,Atlanta Falcons,19.4,26.1,0.407,0.345,27.4,18.4,-14.5,-9.0,Buffalo Bills,Atlanta Falcons
3,17,7,16,Cincinnati Bengals,27.6,20.8,0.459,0.614,Kansas City Chiefs,28.2,21.4,0.472,0.505,24.5,24.5,3.5,0.0,Kansas City Chiefs,Cincinnati Bengals
4,17,31,20,Tennessee Titans,24.6,20.8,0.453,0.566,Miami Dolphins,20.1,21.9,0.485,0.379,23.2,20.5,-3.0,-3.0,Tennessee Titans,Miami Dolphins
5,17,14,17,Indianapolis Colts,26.5,21.5,0.566,0.523,Las Vegas Raiders,22.0,25.8,0.438,0.512,26.2,21.8,-8.5,-4.0,Indianapolis Colts,Las Vegas Raiders
6,17,22,15,New England Patriots,25.7,19.3,0.581,0.528,Jacksonville Jaguars,15.1,26.6,0.408,0.470,26.2,17.2,-17.0,-9.0,New England Patriots,Jacksonville Jaguars
7,17,25,30,New York Jets,19.5,28.4,0.534,0.544,Tampa Bay Buccaneers,30.1,20.8,0.467,0.443,20.2,29.2,14.5,9.0,Tampa Bay Buccaneers,New York Jets
8,17,32,26,Washington Football Team,19.7,25.5,0.597,0.599,Philadelphia Eagles,26.1,22.6,0.412,0.348,21.2,25.8,6.0,5.0,Philadelphia Eagles,Washington Football Team
9,17,3,19,Baltimore Ravens,22.8,23.1,0.605,0.635,Los Angeles Rams,27.1,21.9,0.461,0.413,22.4,25.1,7.0,3.0,Los Angeles Rams,Baltimore Ravens


In [ ]:
## historical results - ready for analysis

In [25]:
home_points_for_wk = []
home_points_for = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['HomeTeamId']==i) & (games['Week'] <= j)][['HomeTeamId','HomeTeamScore']]
        df['WeekRun'] = j
        home_points_for.append(df)
home_points_for_wk.append(pd.concat(home_points_for))
all_home_points_for_wk = pd.concat(home_points_for_wk)
home_points_for_wk = all_home_points_for_wk.groupby(['HomeTeamId','WeekRun'])['HomeTeamScore'].sum().reset_index(name = 'HomePointsFor')

away_points_for_wk = []
away_points_for = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['AwayTeamId']==i) & (games['Week'] <= j)][['AwayTeamId','AwayTeamScore']]
        df['WeekRun'] = j
        away_points_for.append(df)
away_points_for_wk.append(pd.concat(away_points_for))
all_away_points_for_wk = pd.concat(away_points_for_wk)
away_points_for_wk = all_away_points_for_wk.groupby(['AwayTeamId','WeekRun'])['AwayTeamScore'].sum().reset_index(name = 'AwayPointsFor')

away_points_against_wk = []
away_points_against = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['AwayTeamId']==i) & (games['Week'] <= j)][['AwayTeamId','HomeTeamScore']]
        df['WeekRun'] = j
        away_points_against.append(df)
away_points_against_wk.append(pd.concat(away_points_against))
all_away_points_against_wk = pd.concat(away_points_against_wk)
away_points_against_wk = all_away_points_against_wk.groupby(['AwayTeamId','WeekRun'])['HomeTeamScore'].sum().reset_index(name = 'AwayPointsAgainst')

home_points_against_wk = []
home_points_against = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['HomeTeamId']==i) & (games['Week'] <= j)][['HomeTeamId','AwayTeamScore']]
        df['WeekRun'] = j
        home_points_against.append(df)
home_points_against_wk.append(pd.concat(home_points_against))
all_home_points_against_wk = pd.concat(home_points_against_wk)
home_points_against_wk = all_home_points_against_wk.groupby(['HomeTeamId','WeekRun'])['AwayTeamScore'].sum().reset_index(name = 'HomePointsAgainst')

home_games = games[['Week','HomeTeamId']]
home_games = home_games.rename(columns={'HomeTeamId': 'TeamId'})
away_games = games[['Week','AwayTeamId']]
away_games = away_games.rename(columns={'AwayTeamId': 'TeamId'})
games2 = pd.concat([home_games, away_games], axis=0)
games2 = games2.sort_values(['TeamId','Week'], ascending = [True,True])
home_points_full_wk = pd.merge(home_points_for_wk, home_points_against_wk, how = 'inner', on = ['HomeTeamId','WeekRun'])
away_points_full_wk = pd.merge(away_points_for_wk, away_points_against_wk, how = 'inner', on = ['AwayTeamId','WeekRun'])
home_points_full_wk = home_points_full_wk.rename(columns={'HomeTeamId': 'TeamId'})
away_points_full_wk = away_points_full_wk.rename(columns={'AwayTeamId': 'TeamId'})
home_points_full_wk2 = pd.merge(games2, home_points_full_wk, how = 'left', left_on = ['TeamId','Week'], right_on = ['TeamId','WeekRun'])
away_points_full_wk2 = pd.merge(games2, away_points_full_wk, how = 'left', left_on = ['TeamId','Week'], right_on = ['TeamId','WeekRun'])

points_wk = pd.merge(home_points_full_wk2[['Week','TeamId','HomePointsFor','HomePointsAgainst']], away_points_full_wk2[['Week','TeamId','AwayPointsFor','AwayPointsAgainst']], how = 'inner', on = ['Week','TeamId'])
points_wk['TotalPointsFor'] = np.where(points_wk['HomePointsFor'].notnull(), points_wk['HomePointsFor'], 0) + np.where(points_wk['AwayPointsFor'].notnull(), points_wk['AwayPointsFor'], 0)
points_wk['TotalPointsAgainst'] = np.where(points_wk['HomePointsAgainst'].notnull(), points_wk['HomePointsAgainst'], 0) + np.where(points_wk['AwayPointsAgainst'].notnull(), points_wk['AwayPointsAgainst'], 0)

points_wk.head()

,Week,TeamId,HomePointsFor,HomePointsAgainst,AwayPointsFor,AwayPointsAgainst,TotalPointsFor,TotalPointsAgainst
0,1,1,NaN,NaN,38.0,13.0,38.0,13.0
1,2,1,34.0,33.0,38.0,13.0,72.0,46.0
2,3,1,34.0,33.0,69.0,32.0,103.0,65.0
3,4,1,34.0,33.0,106.0,52.0,140.0,85.0
4,5,1,51.0,43.0,106.0,52.0,157.0,95.0


In [28]:
week_wins = []
wins = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['WinningTeamId']==i) & (games['Week']<=j)][['WinningTeamId','LosingTeamId']]
        df['WeekRun'] = j
        wins.append(df)
week_wins.append(pd.concat(wins))
all_wins = pd.concat(week_wins)
all_wins_wk = all_wins.groupby(['WinningTeamId','WeekRun'])['LosingTeamId'].count().reset_index(name='Wins')

week_losses = []
losses = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['LosingTeamId']==i) & (games['Week']<=j)][['WinningTeamId','LosingTeamId']]
        df['WeekRun'] = j
        losses.append(df)
week_losses.append(pd.concat(losses))
all_losses = pd.concat(week_losses)
all_losses_wk = all_losses.groupby(['LosingTeamId','WeekRun'])['WinningTeamId'].count().reset_index(name='Losses')

points_record_wk = pd.merge(points_wk, all_wins_wk, how = 'left', left_on = ['TeamId', 'Week'], right_on = ['WinningTeamId', 'WeekRun'])
points_record_wk = pd.merge(points_record_wk, all_losses_wk, how = 'left', left_on = ['TeamId', 'Week'], right_on = ['LosingTeamId', 'WeekRun'])

# still missing weeks here
points_record_wk2 = points_record_wk[['Week','TeamId','Wins','Losses','TotalPointsFor','TotalPointsAgainst','HomePointsFor','HomePointsAgainst','AwayPointsFor','AwayPointsAgainst']]
points_record_wk2['Wins'] = np.where(points_record_wk['Wins'].notnull(), points_record_wk['Wins'], 0)
points_record_wk2['Losses'] = np.where(points_record_wk['Losses'].notnull(), points_record_wk['Losses'], 0)
points_record_wk2['Pct'] = round(points_record_wk2['Wins'] / (points_record_wk2['Wins'] + points_record_wk2['Losses']),3)
points_record_wk2['AvgPointsFor'] = round(points_record_wk2['TotalPointsFor'] / (points_record_wk2['Wins'] + points_record_wk2['Losses']),1)
points_record_wk2['AvgPointsAgainst'] = round(points_record_wk2['TotalPointsAgainst'] / (points_record_wk2['Wins'] + points_record_wk2['Losses']),1)

week_idx = pd.MultiIndex.from_product((range(1,max(games['Week'])),range(1,33)), names=['Week','TeamId'])
points_record_wk2 = points_record_wk2.set_index(['Week','TeamId']).reindex(week_idx).reset_index()
points_record_wk2 = points_record_wk2.sort_values(['TeamId','Week'], ascending = [True,True])

points_record_wk2['Wins'] = points_record_wk2.groupby(['TeamId'])['Wins'].ffill(limit=1)
points_record_wk2['Losses'] = points_record_wk2.groupby(['TeamId'])['Losses'].ffill(limit=1)
points_record_wk2['TotalPointsFor'] = points_record_wk2.groupby(['TeamId'])['TotalPointsFor'].ffill(limit=1)
points_record_wk2['TotalPointsAgainst'] = points_record_wk2.groupby(['TeamId'])['TotalPointsAgainst'].ffill(limit=1)
points_record_wk2['HomePointsFor'] = points_record_wk2.groupby(['TeamId'])['HomePointsFor'].ffill(limit=1)
points_record_wk2['HomePointsAgainst'] = points_record_wk2.groupby(['TeamId'])['HomePointsAgainst'].ffill(limit=1)
points_record_wk2['AwayPointsFor'] = points_record_wk2.groupby(['TeamId'])['AwayPointsFor'].ffill(limit=1)
points_record_wk2['AwayPointsAgainst'] = points_record_wk2.groupby(['TeamId'])['AwayPointsAgainst'].ffill(limit=1)
points_record_wk2['Pct'] = points_record_wk2.groupby(['TeamId'])['Pct'].ffill(limit=1)
points_record_wk2['AvgPointsFor'] = points_record_wk2.groupby(['TeamId'])['AvgPointsFor'].ffill(limit=1)
points_record_wk2['AvgPointsAgainst'] = points_record_wk2.groupby(['TeamId'])['AvgPointsAgainst'].ffill(limit=1)

points_record_wk2.sort_values(['TeamId','Week'], ascending = [True,True]).head()

<ipython-input-28-f57afa3e9918>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_record_wk2['Wins'] = np.where(points_record_wk['Wins'].notnull(), points_record_wk['Wins'], 0)
<ipython-input-28-f57afa3e9918>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_record_wk2['Losses'] = np.where(points_record_wk['Losses'].notnull(), points_record_wk['Losses'], 0)
<ipython-input-28-f57afa3e9918>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,Week,TeamId,Wins,Losses,TotalPointsFor,TotalPointsAgainst,HomePointsFor,HomePointsAgainst,AwayPointsFor,AwayPointsAgainst,Pct,AvgPointsFor,AvgPointsAgainst
0,1,1,1.0,0.0,38.0,13.0,NaN,NaN,38.0,13.0,1.0,38.0,13.0
32,2,1,2.0,0.0,72.0,46.0,34.0,33.0,38.0,13.0,1.0,36.0,23.0
64,3,1,3.0,0.0,103.0,65.0,34.0,33.0,69.0,32.0,1.0,34.3,21.7
96,4,1,4.0,0.0,140.0,85.0,34.0,33.0,106.0,52.0,1.0,35.0,21.2
128,5,1,5.0,0.0,157.0,95.0,51.0,43.0,106.0,52.0,1.0,31.4,19.0


In [93]:
# new index
#dates = pd.to_datetime(pd.date_range('01-02-2019', '06-30-2020'))
week_idx = pd.MultiIndex.from_product((range(1,max(games['Week'])),range(1,33)), names=['Week','TeamId'])
all_losses_wk_pct_full = all_losses_wk_pct2[['Week','TeamId','SOL']].set_index(['Week','TeamId']).reindex(week_idx).reset_index()
all_wins_wk_pct_full = all_wins_wk_pct2[['Week','TeamId','SOW']].set_index(['Week','TeamId']).reindex(week_idx).reset_index()
all_losses_wk_pct_full['SOL'] = all_losses_wk_pct_full.groupby(['TeamId', 'TeamId'])['SOL'].ffill(limit=1)
all_wins_wk_pct_full['SOW'] = all_wins_wk_pct_full.groupby(['TeamId', 'TeamId'])['SOW'].ffill(limit=1)
all_losses_wk_pct_full[all_losses_wk_pct_full['TeamId'] == 31]

,Week,TeamId,SOL
30,1,31,1.000
62,2,31,1.000
94,3,31,1.000
126,4,31,0.625
158,5,31,0.600
190,6,31,1.000
222,7,31,0.584
254,8,31,0.580
286,9,31,0.570


In [27]:
# Rolling SOW and SOL

wins_wk = []
wins = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['WinningTeamId']==i) & (games['Week'] <= j)][['WinningTeamId','LosingTeamId']]
        df['WeekRun'] = j
        wins.append(df)
wins_wk.append(pd.concat(wins))
all_wins_wk = pd.concat(wins_wk)
all_wins_wk = all_wins_wk.sort_values(['WinningTeamId','WeekRun'], ascending = [True,True])
all_wins_wk_pct = pd.merge(all_wins_wk, points_record_wk2[['TeamId','Week','Pct']], how = 'inner', left_on = ['LosingTeamId','WeekRun'], right_on = ['TeamId','Week'])
all_wins_wk_pct2 = all_wins_wk_pct.groupby(['WinningTeamId','Week'])['Pct'].mean().round(3).reset_index(name = 'SOW')
all_wins_wk_pct2 = all_wins_wk_pct2.rename(columns={'WinningTeamId': 'TeamId'})

losses_wk = []
losses = []
for j in range(1,max(games['Week'])):
    for i in range(1,33):
        df = games[(games['LosingTeamId']==i) & (games['Week'] <= j)][['WinningTeamId','LosingTeamId']]
        df['WeekRun'] = j
        losses.append(df)
losses_wk.append(pd.concat(losses))
all_losses_wk = pd.concat(losses_wk)
all_losses_wk = all_losses_wk.sort_values(['LosingTeamId','WeekRun'], ascending = [True,True])
all_losses_wk_pct = pd.merge(all_losses_wk, points_record_wk2[['TeamId','Week','Pct']], how = 'inner', left_on = ['WinningTeamId','WeekRun'], right_on = ['TeamId','Week'])
all_losses_wk_pct2 = all_losses_wk_pct.groupby(['LosingTeamId','Week'])['Pct'].mean().round(3).reset_index(name = 'SOL')
all_losses_wk_pct2 = all_losses_wk_pct2.rename(columns={'LosingTeamId': 'TeamId'})

week_idx = pd.MultiIndex.from_product((range(1,max(games['Week'])),range(1,33)), names=['Week','TeamId'])
all_losses_wk_pct_full = all_losses_wk_pct2[['Week','TeamId','SOL']].set_index(['Week','TeamId']).reindex(week_idx).reset_index()
all_wins_wk_pct_full = all_wins_wk_pct2[['Week','TeamId','SOW']].set_index(['Week','TeamId']).reindex(week_idx).reset_index()
all_losses_wk_pct_full['SOL'] = all_losses_wk_pct_full.groupby(['TeamId', 'TeamId'])['SOL'].ffill(limit=1)
all_wins_wk_pct_full['SOW'] = all_wins_wk_pct_full.groupby(['TeamId', 'TeamId'])['SOW'].ffill(limit=1)

points_record_wk3 = pd.merge(points_record_wk2, all_wins_wk_pct_full, how = 'left', on = ['TeamId','Week'])
points_record_wk3 = pd.merge(points_record_wk3, all_losses_wk_pct_full, how = 'left', on = ['TeamId','Week'])
points_record_wk3.sort_values(['TeamId','Week'], ascending = [True,True])
points_record_wk3['AsOfWeek'] = points_record_wk3['Week'] + 1
points_record_wk3[points_record_wk3['AsOfWeek']==2].head()

,Week,TeamId,Wins,Losses,TotalPointsFor,TotalPointsAgainst,HomePointsFor,HomePointsAgainst,AwayPointsFor,AwayPointsAgainst,Pct,AvgPointsFor,AvgPointsAgainst,SOW,SOL,AsOfWeek
0,1,1,1.0,0.0,38.0,13.0,NaN,NaN,38.0,13.0,1.0,38.0,13.0,0.0,NaN,2
17,1,2,0.0,1.0,6.0,32.0,6.0,32.0,NaN,NaN,0.0,6.0,32.0,NaN,1.0,2
34,1,3,0.0,1.0,27.0,33.0,NaN,NaN,27.0,33.0,0.0,27.0,33.0,NaN,1.0,2
51,1,4,0.0,1.0,16.0,32.0,16.0,32.0,NaN,NaN,0.0,16.0,32.0,NaN,1.0,2
68,1,5,1.0,0.0,19.0,14.0,19.0,14.0,NaN,NaN,1.0,19.0,14.0,0.0,NaN,2


In [23]:
games_hist_home = pd.merge(games, points_record_wk3[['TeamId','AsOfWeek','AvgPointsFor','AvgPointsAgainst','SOW','SOL']], how = 'inner', left_on = ['Week','HomeTeamId'], right_on = ['AsOfWeek','TeamId'])
games_hist_home = games_hist_home.rename(columns={'AvgPointsFor': 'AvgPointsForH', 'AvgPointsAgainst': 'AvgPointsAgainstH', 'SOW': 'SOWH', 'SOL': 'SOLH'})
games_hist_home = games_hist_home.drop(games_hist_home.columns[[0,12]], axis=1)
games_hist_home = pd.merge(games_hist_home, teams[['Id','Team']], how = 'inner', left_on = ['HomeTeamId'], right_on = ['Id'])
games_hist_home = games_hist_home.rename(columns={'Team': 'HomeTeam'})
games_hist_home = games_hist_home.drop(games_hist_home.columns[16], axis=1)

games_hist_away = pd.merge(games, points_record_wk3[['TeamId','AsOfWeek','AvgPointsFor','AvgPointsAgainst','SOW','SOL']], how = 'inner', left_on = ['Week','AwayTeamId'], right_on = ['AsOfWeek','TeamId'])
games_hist_away = games_hist_away.rename(columns={'AvgPointsFor': 'AvgPointsForA', 'AvgPointsAgainst': 'AvgPointsAgainstA', 'SOW': 'SOWA', 'SOL': 'SOLA'})
games_hist_away = games_hist_away.drop(games_hist_away.columns[[0,12]], axis=1)
games_hist_away = pd.merge(games_hist_away, teams[['Id','Team']], how = 'inner', left_on = ['AwayTeamId'], right_on = ['Id'])
games_hist_away = games_hist_away.rename(columns={'Team': 'AwayTeam'})
games_hist_away = games_hist_away.drop(games_hist_away.columns[16], axis=1)

games_hist_full = pd.merge(games_hist_home, games_hist_away[['AsOfWeek','HomeTeamId','AwayTeamId','AwayTeam','AvgPointsForA','AvgPointsAgainstA','SOWA','SOLA']], how = 'inner', on = ['AsOfWeek','HomeTeamId','AwayTeamId'])
games_hist_full['ExpectedPtsH'] = round((games_hist_full['AvgPointsForH'] + games_hist_full['AvgPointsAgainstA'])/2,1)
games_hist_full['ExpectedPtsA'] = round((games_hist_full['AvgPointsForA'] + games_hist_full['AvgPointsAgainstH'])/2,1)
games_hist_full['ExpectedSpread'] = round(games_hist_full['ExpectedPtsA'],0) - round(games_hist_full['ExpectedPtsH'],0)
games_hist_full['ExpectedWinner'] = np.where(games_hist_full['ExpectedPtsH'] > games_hist_full['ExpectedPtsA'], games_hist_full['HomeTeam'], games_hist_full['AwayTeam'])
games_hist_full['ExpectedCover'] = np.where((games_hist_full['Spread'] < 0) & (games_hist_full['ExpectedSpread'] < games_hist_full['Spread']), games_hist_full['HomeTeam']
                                       , np.where((games_hist_full['Spread'] < 0) & (games_hist_full['ExpectedSpread'] >= games_hist_full['Spread']), games_hist_full['AwayTeam']
                                       , np.where((games_hist_full['Spread'] > 0) & (games_hist_full['ExpectedSpread'] <= games_hist_full['Spread']), games_hist_full['HomeTeam']
                                       , np.where((games_hist_full['Spread'] > 0) & (games_hist_full['ExpectedSpread'] > games_hist_full['Spread']), games_hist_full['AwayTeam']
                                       , np.where((games_hist_full['Spread'] == 0) & (games_hist_full['ExpectedSpread'] > 0), games_hist_full['AwayTeam']
                                       , np.where((games_hist_full['Spread'] == 0) & (games_hist_full['ExpectedSpread'] < 0), games_hist_full['HomeTeam'], np.NaN))))))

games_hist_full['ActualCover'] = np.where((games_hist_full['Spread'] < 0) & (games_hist_full['AwayTeamScore'] - games_hist_full['HomeTeamScore'] < games_hist_full['Spread']), games_hist_full['HomeTeam']
                                       , np.where((games_hist_full['Spread'] < 0) & (games_hist_full['AwayTeamScore'] - games_hist_full['HomeTeamScore'] >= games_hist_full['Spread']), games_hist_full['AwayTeam']
                                       , np.where((games_hist_full['Spread'] > 0) & ((games_hist_full['HomeTeamScore'] - games_hist_full['AwayTeamScore'] <= games_hist_full['Spread']) | (games_hist_full['HomeTeamScore'] > games_hist_full['AwayTeamScore'])), games_hist_full['HomeTeam']
                                       , np.where((games_hist_full['Spread'] > 0) & (games_hist_full['AwayTeamScore'] - games_hist_full['HomeTeamScore'] > games_hist_full['Spread']), games_hist_full['AwayTeam']
                                       , np.where((games_hist_full['Spread'] == 0) & (games_hist_full['AwayTeamScore'] - games_hist_full['HomeTeamScore'] > 0), games_hist_full['AwayTeam']
                                       , np.where((games_hist_full['HomeTeamScore'] - games_hist_full['AwayTeamScore'] < 0), games_hist_full['HomeTeam'], np.NaN))))))

games_hist_full['WinningTeamName'] = np.where(games_hist_full['WinningTeam'] =='H', games_hist_full['HomeTeam'], games_hist_full['AwayTeam'])
games_hist_full['WinnerMatch'] = np.where(games_hist_full['ExpectedWinner'] == games_hist_full['WinningTeamName'], 'Y', 'N')
games_hist_full['CoverMatch'] = np.where(games_hist_full['ExpectedCover'] == games_hist_full['ActualCover'], 'Y', 'N')
games_hist_full['HomeScoreDiff'] = games_hist_full['ExpectedPtsH'] - games_hist_full['HomeTeamScore']
games_hist_full['AwayScoreDiff'] = games_hist_full['ExpectedPtsA'] - games_hist_full['AwayTeamScore']
games_hist_full['SpreadDiff'] = games_hist_full['ExpectedSpread'] - games_hist_full['Spread']

games_hist_full_no_wk1 = games_hist_full[games_hist_full['Week'] > 1]
games_hist_full_no_wk1.head()

,Week,HomeTeamId,AwayTeamId,Spread,HomeTeamScore,AwayTeamScore,Result,ScoreDiff,WinningTeamId,WinningTeam,...,ExpectedSpread,ExpectedWinner,ExpectedCover,ActualCover,WinningTeamName,WinnerMatch,CoverMatch,HomeScoreDiff,AwayScoreDiff,SpreadDiff
0,2,32,24,-3.0,30.0,29.0,W,1.0,32.0,H,...,-6.0,Washington Football Team,Washington Football Team,New York Giants,Washington Football Team,Y,N,-8.5,-12.5,-3.0
1,5,32,23,2.0,22.0,33.0,L,-11.0,23.0,A,...,6.0,New Orleans Saints,New Orleans Saints,Washington Football Team,New Orleans Saints,Y,N,-0.8,-6.0,4.0
2,6,32,16,6.5,13.0,31.0,L,-18.0,16.0,A,...,2.0,Kansas City Chiefs,Washington Football Team,Washington Football Team,Kansas City Chiefs,Y,Y,15.6,-0.1,-4.5
3,10,32,30,9.5,NaN,NaN,T,NaN,NaN,nan,...,9.0,Tampa Bay Buccaneers,Washington Football Team,NaN,Tampa Bay Buccaneers,Y,N,NaN,NaN,-0.5
4,2,18,9,-3.0,17.0,20.0,L,-3.0,9.0,A,...,-4.0,Los Angeles Chargers,Los Angeles Chargers,Dallas Cowboys,Dallas Cowboys,N,N,8.5,2.5,-1.0
